# Visualización 3 y 4 de métrica precios

In [52]:
import plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [53]:
# Lectura de los datos a utilizar
df = pd.read_pickle("predicciones.pkl")

## Funciones

In [43]:
def add_porcentual(p_df, a = True):
    """
    Función que hace el cálculo del cambio porcentual en precios y que adecua el df para la realización de las tablas.

    Parameters
    ---------
    p_df: DataFrame : df que contiene el último precio y el precio estimado a 6 meses por grupos y clases.
    
    Returns
    ---------
    df:  DataFrame : df que contiene lo que el anterior y el cambio porcentual de los precios.
    
    Debuggin
    ---------
    p_df = add_procentual(df)
  
    """
    p_df['C_Porcentual']=round(p_df['Precio para Nov 2020']/df['Ultimo precio']-1, 4)
    p_df.reset_index(level=1, inplace=True)
    p_df.reset_index(level=0, inplace=True)
    p_df = p_df.rename(columns={'index' :'Grupo', 'level_1':'Clase'})
    return p_df

In [44]:
def add_color(p_df):
    """
    Función que asigna el color segun el cambio porcentual.

    Parameters
    ---------
    p_df: DataFrame : df que contiene el último precio, el precio estimado a 6 meses y el cambio porcentual por grupos y clases.
    
    Returns
    ---------
    df:  DataFrame : df que contiene lo que el anterior y la asignación de colores.
    
    Debuggin
    ---------
    p_df = clean_df(df)
  
    """
    
    color = []
    for i in range(0, len(p_df)):
        if p_df['C_Porcentual'][i]<=-.01:
            color.append('Precios que bajaran más de 1%')
        if p_df['C_Porcentual'][i]<=.01 and df['C_Porcentual'][i]>-.01:
            color.append('Precios que se mantendrán con una variación menor a 1%')
        if p_df['C_Porcentual'][i]>.01:
            color.append('Precios que aumentarán más de 1%')
    p_df['Color']=color
    return p_df

In [45]:
def treemap_chart(p_df, path, color=[]):
    """
    Función que crea la tabla treemap

    Parameters
    ---------
    p_df: DataFrame : df que contiene los precios, su cambio y el color asignado.
    path: list : lista que contiene los labels a mostrar en la tabla.
    color: list : lista que contiene los colores a mostrar. 
    
    Returns
    ---------
    fig: chart: Tabla que se solicita.
    
    Debuggin
    ---------
    path = ['Grupo','Clase', 'C_Porcentual']
    color = ["yellow", "red", "green"]
    p_df = treemap_chart(p_df, path, color)
  
    """
    fig = px.treemap(p_df, path=path)
    fig.update_layout(treemapcolorway = color)
    fig.update_traces(hovertemplate= '<b>%{label}')
    fig.update_layout(title_text='Cambios Porcentuales en los precios por Grupo')
    return fig.show()

In [46]:
def div_grupo(p_df, indice):
    """
    Función que hace un groupby por grupo.

    Parameters
    ---------
    p_df: DataFrame : df que contiene los precios, su cambio y el color asignado.
    indice: int: número de grupo que se desea obtener. (0-14)
    
    Returns
    ---------
    grupo: df : contiene los datos del grupo seleccionado unicamente.
    cat: str : nombre de la categoria que se esta utiizando.
    
    Debuggin
    ---------
   
    grupo = div_grupo(df, 4)
  
    """
    gk = df.groupby('Grupo')
    d_keys = gk.groups
    categories = list(d_keys.keys())
    cat = categories[indice]
    grupo = gk.get_group(cat)
    return grupo, cat

In [47]:
def bar_chart_sem(grupo, cat):
    """
    Función que crea la tabla de barras en forma de semaforo.

    Parameters
    ---------
    grupo: df : contiene los valores unicamente del grupo seleccionado (precio, cambio, color)
    cat: str : contiene el nombre de la categoria utilizada.
    
    Returns
    ---------
    fig: chart: Tabla que se solicita.
    
    Debuggin
    ---------
    
    bar_chart_sem(grupo,cat)
  
    """

    colors = list()
    x = list(grupo['Clase'])
    y = list(grupo['C_Porcentual'])
    data = list(grupo['C_Porcentual'])
    for i in range(0, len(grupo)):
        if data[i]<=-.01:
            colors.append('green')
        if data[i]<=.01 and data[i]>-.01:
            colors.append('yellow')
        if data[i]>.01:
            colors.append('red')

    fig = go.Figure(data=[go.Bar(
        x=x,
        y=y,
        marker_color=colors # marker color can be a single color value or an iterable
    )])
    fig.update_layout(title_text=cat, yaxis_range=(-.06,.05))
    return fig.show()
    

## Principal 

In [54]:
#Para correr treemap de colores
df = add_porcentual(df)
path = ['Grupo','Clase', 'C_Porcentual']
treemap_chart(df, path)

In [49]:
#Para correr treemap semaforo
df = add_color(df)
path = ['Color','Grupo','Clase', 'C_Porcentual']
color = ['yellow', 'green', 'red']
label = df['Clase']
df['C_Porcentual']= df['C_Porcentual'].apply(lambda x:' Cambio Porcentual: ' + str(x))
treemap_chart(df, path, color)

In [55]:
#Para correr gráfica de barras semáforo con filtro
grupo, cat = div_grupo(df, 2)
bar_chart_sem(grupo,cat)